# XX-Cyg : Time Series analysis avec TESS data
### June Parsons | 20191216

---

![XX-Cyg ](tangerine_smooth_slick_yum_lc_xxcyg.png)

---



# Dependencies 
---

DRACO currently runs off of the following dependencies: 

#### Numpy ()
#### Matplotlib ()
#### Photutils ()
#### Astropy ()
#### Astroquery () *
#### ccdproc ()
#### Scipy () *
#### Scikit/Skimage () *
#### Sklearn () *

We'd like to extend immense gratitude to the teams behind these wonderful opensource projects; without their contribution DRACO would not be able to function. We would also like to thank the Transiting exoplanet survey satellite (TESS) team for their dedication and thorough open access data.

# Proposed release naming scheme

---

#### 1. Apogee (Alpha release) Current
#### 2. Bolide (Beta release) Late December 2019
#### 3. Callisto (Preview release) Early January 2020
#### 4. DRACO (First public release - working reading and deployable) First Quarter 2020

## Extended release map
5. Erin (Release of lazy eye)
6. Fleming Faber (Release of telescope and observing tools [tracking & focus analysis], dust covers, Photometry calibration)
7. Ganymede (Release of Astrophoto tools [RVB reduction & stacking])
8. Henrietta (Front end interface testing & observation planning)
9. Iridium (Human readable data output and reporting)
10. Jovian (Front end interface release)
11. Kepler
12. Laniakea
13. Misato
14. Nexus/NERV (Data sharing capabilities, results upload, possible database storage/access)
15. Oort (Working with moving objects, parallax, asteroid monitoring)
16. Payne (Multi filter colour profiling)
17. Quanta (Small release?) (Stable long term release)
18. Robin
19. Sirius (Working with spectra?)
20. Taurus (Telescope drive and control updates)
21. Uranus (Utilities update)
22. Valhalla
23. Whipper snipper (Spiral galaxy analysis, spin vector, tilt, interaction, distance, activity, and classification)
24. Xerneas
25. Yveltal
26. Zelda (Stable long term release)

---

# Possible future features

---

1. (Using lightcurves to generate physical models) Kepler?

# Non-code related next steps:
---
Next steps include running script through all of TESS XXCyg data and saving times of max light. Comparing the time of max light to data obtained at AICO. Running Fourier analysis through ~18000 historic data points, obtaining times of max light and comparing to the properties of all other data sources.

Planning for future TESS data.

In [1]:
# %matplotlib notebook
%matplotlib ipympl
# Import to clean up package warning information about future depreciation
# Import to keep track of computing time
import datetime
import warnings
warnings.filterwarnings('ignore')
#########
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import lightkurve as lk
from lightkurve.lightcurve import TessLightCurve as tlc
from photutils import aperture_photometry, RectangularAperture, RectangularAnnulus
import dracoOP2 as dr2
# from sklearn.preprocessing import normalize

# from __future__ import braces
# import this
# import __hello__

In [2]:
# Initialize time tracker variables - and print start time
START_DATE_TIME = datetime.datetime.now()

print('\nStarting time: ', START_DATE_TIME)


Starting time:  2020-03-02 20:00:10.357130


In [3]:
# Open the TPF and print important info 
# fits_file = "tess2019331140908-s0019-0000000392774261-0164-s_tp.fits" #bl-cam
fits_file = "tess2019253231442-s0016-0000000233310793-0152-s_tp.fits" #xx cyg file 1
# fits_file = "tess2019198215352-s0014-0000000233310793-0150-s_tp.fits" #xx cyg file 2
# fits_file = "tess2019279210107-s0017-0000000233310793-0161-s_tp.fits" #xx cyg file 3
# fits_file = "tess2019226182529-s0015-0000000233310793-0151-s_tp.fits" #xx cyg file 4

fits.info(fits_file) # Print out file info for the target pixel file
fits.getdata(fits_file, ext=1).columns
tpf = lk.open(fits_file)

Filename: tess2019253231442-s0016-0000000233310793-0152-s_tp.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      44   ()      
  1  PIXELS        1 BinTableHDU    248   17765R x 11C   [D, E, J, 121J, 121E, 121E, 121E, 121E, J, E, E]   
  2  APERTURE      1 ImageHDU        49   (11, 11)   int32   
  3  TARGET COSMIC RAY    1 BinTableHDU     27   0R x 4C   [J, I, I, E]   


In [4]:
# Store important contents of the TPF as separate arrays in readonly
with fits.open(fits_file, mode="readonly") as hdulist:
    tess_bjds = hdulist[1].data['TIME']
    raw_counts = hdulist[1].data['RAW_CNTS']
    calibrated_fluxes = hdulist[1].data['FLUX']
    flux_err = hdulist[1].data['FLUX_ERR']
# Store the TID 
tid = 'TIC 233310793'

In [5]:
# make storage arrays for aperture results and flux error per stamp
aperture_sums = []
err = []
# Iterate over each stamp in the TPF
for s in range(calibrated_fluxes.shape[0]):
        # Construct a custom aperture and annulus 
        aperture = RectangularAperture((4,4), 3,3) # blcam
        annulus_aperture = RectangularAnnulus((4,4), 4, 5, 5) # blcam
#         aperture = RectangularAperture((5,4), 3,3) # file 2
#         annulus_aperture = RectangularAnnulus((5,4), 4, 5, 5) # file 2
#         aperture = RectangularAperture((6,5), 3,3)
#         annulus_aperture = RectangularAnnulus((6,5), 4, 5, 5)
        # Combine the aperture
        aps = [aperture , annulus_aperture]
        # Place the aperature on the current stamp and perform aperture photometry
        phot_table = aperture_photometry(calibrated_fluxes[s,:,:], aps)
        # Compute the background values from the annulus photometry
        bkg_sum = (phot_table['aperture_sum_1'] / annulus_aperture.area) * aperture.area
        # Subtract the background from main aperture
        phot_table['residual_aperture_sum'] = phot_table['aperture_sum_0'] - bkg_sum
        # Store the aperture results 
        aperture_sums.append(float(phot_table['aperture_sum_0']) - float(bkg_sum))
        # Store the flux error
        err.append(np.mean(flux_err[s,:,:]))
 

In [6]:
# Print the first aperture value to compare with expectation
print(aperture_sums[0])
# Construct a lightcurve object using the computed series
lc_june = lk.TessLightCurve(time = tess_bjds, flux = np.array(aperture_sums), flux_err = err, flux_unit = None, time_format = 'btjd', time_scale = 'tdb', centroid_col = None, centroid_row = None, quality = None, quality_bitmask = None, cadenceno = None, sector = tpf.sector, camera = tpf.camera, ccd = tpf.ccd, targetid = 233310793, ra = tpf.ra, dec = tpf.dec, label = 'XX-CYG')

#print(len(tess_bjds), len(aperture_sums), len(err))
#print(lc_june.flux.shape, lc_june.time.shape, lc_june.flux_err.shape)

# Print the new lightcurve object properties for record
lc_june.show_properties()
# Clean the series of outliers > 7 sigma from the series
lc_june = lc_june.remove_outliers(sigma=7)
# Remove the downward flux trend expirienced by Tess throught an orbit
lc_june = lc_june.flatten(window_length=1001)
lc_june = lc_june.normalize()

# Produce a scatter with error bar plot of the lightcurve
# ax = lc_june.errorbar(label = 'Error', title = 'XXCyg Lightcurve With Flux Error : Normalized')
# ax = ax,
# lc_june.scatter(s = 0.1, label = 'XXCyg lightcurve', c=lc_june.flux, cmap='autumn', title = 'XXCyg Lightcurve With Flux Error : Normalized')
            



2286.3850288391113
   Attribute              Description           
--------------- --------------------------------
         camera                                2
            ccd                                2
         sector                               16
       targetid                        233310793
          label                           XX-CYG
        mission                             TESS
    time_format                             btjd
     time_scale                              tdb
      cadenceno                   array (17765,)
   centroid_col                   array (17765,)
   centroid_row                   array (17765,)
           flux                   array (17765,)
       flux_err                   array (17765,)
  flux_quantity                   array (17765,)
        quality                   array (17765,)
           time                   array (17765,)
   astropy_time <class 'astropy.time.core.Time'>
            dec                  <class 'float'>
 

In [7]:
# Store the lightcurve flux column
aperature_sums = lc_june.flux 

# Construct Astropy Table column objects that contain the flux values and their associated times
c1 = fits.Column(name='APERTURE_SUM', array = aperture_sums, format='D')
c2 = fits.Column(name='TIME', array = tess_bjds, format='D')
# Combine the columns into a fits HDU (Header Data Unit)
table_hdu = fits.BinTableHDU.from_columns([c1, c2])

# Produce the fits Header
hdr = fits.Header()
# Set the fits Header as the Primary HDU
h_primary = fits.PrimaryHDU(header=hdr)
# Combine into a single fits object
hdul = fits.HDUList([h_primary, table_hdu])

# Write the fits object to a file in the root of the python pipeline
# hdul.writeto('233310793_june_lc.fits', overwrite = True)

In [8]:
# Start figure and axis.
# fig, ax = plt.subplots()
# print(tess_bjds.shape)
# # Convert times to HJD

# # Let's define a title for the figure.
# fig.suptitle("XX-Cyg Lightcurve - Sector 15, 2 Min Cadence")
# ax.scatter(lc_june.time[0:500], lc_june.flux[0:500], c = 'g', marker = '.', s = 0.5)
# # ax.scatter(tess_bjds, aperture_sums, c = 'g', marker = '.', s = 0.1)
# #ax.grid()
# plt.show()


In [9]:
# Homebuilt Fourier analysis - Fourier fitting and local max recording
# Compute ifft()
x = lc_june.time#[0:4000]
y = lc_june.flux#[0:4000]
# y = y / np.linalg.norm(y)

from scipy.signal import find_peaks, periodogram, spectrogram, lombscargle, welch
from astropy.timeseries import TimeSeries, LombScargle

def fournax(x, y, terms):
    # Fournax is an abbreviation of Fourier numerical astronomy extension, its name is a backronym styled to match the constellation 'fornax'.
    
    # Fourier series domain
    #tau = (np.max(x) - np.min(x))
    # Compute real valued Fourier transform
    f = np.fft.rfft(y)
    # Null or zero coefficients above ammount of series "terms"
    # This corresponds to undesired high-frequency terms
    f[terms+1:] = 0
    # Collapse back into function space, result is smoothed Fourier curve
    F = np.fft.irfft(f)
    
    return F

def pour(y):
    # Fournax is an abbreviation of Fourier numerical astronomy extension, its name is a backronym styled to match the constellation 'fornax'.

    # Compute real valued Fourier transform
    f = np.fft.fft(y)
    # p = np.square(np.abs(f))
    p = np.square(np.abs(f))
    
    return p



terms = 500
Fcurve = fournax(x, y, terms)

terms2 = 550
Fcurve2 = fournax(x, y, terms2)

terms3 = 1000
# terms3 = 5000 # bl cam
Fcurve3 = fournax(x, y, terms3)



In [10]:


# f = np.fft.rfft(y)
# f = f[0:terms3] 
# pgram = lombscargle(x, y, f)
# Pcurve = pgram

# ts = lc_june.to_timeseries()
# fr, Pxx = periodogram(ts)

# frequency, power = LombScargle(x, y).autopower()
# plt.plot(frequency, power) 

# Find times of max and min
Pxx = pour(y) # [0:int(np.floor(len(Fcurve3)/2))]
# Pxx = pour(Fcurve3) # [0:int(np.floor(len(Fcurve3)/2))]

freqs = np.arange(1, len(Pxx), 1)
# timestep in days
timestep = 2 / 1440
n = len(Fcurve3) # [0:int(np.floor(len(Fcurve3)/2))]
freq = np.fft.fftfreq(n, d=timestep)

# Compute mean to find peaks above
h = np.mean(Fcurve3)
hp = np.mean(Pxx)
peaks, _ = find_peaks(Fcurve3, height =  1.1 * h)

# peaks_p, _ = find_peaks(Pxx) # hp ) # 10000 )  # 100) # hp * 0.1)
peaks_p, _ = find_peaks(Pxx, height = hp ) # hp ) # 10000 )  # 100) # hp * 0.1)


# Visualization
fig, ax = plt.subplots()
# Let's define a title for the figure.
# fig.suptitle("BL-Cam Lightcurve - Sector 15, 2 Min Cadence")
fig.suptitle("XX-Cyg Lightcurve - Sector 15, 2 Min Cadence")

ax.plot(x, y, color='lightblue', label = 'raw data', linewidth=0.5)
# ax.plot(x, Fcurve, label = ("'rFFT' series fit (%d terms)" % terms), linewidth=0.5, linestyle='-.')
ax.plot(x[peaks], Fcurve3[peaks], 'x', label = ("'rFFT' series peaks" ))
# ax.plot(x, Fcurve2, label = ("'rFFT' series fit (%d terms)" % terms2), linewidth=0.5, linestyle='--')
ax.plot(x, Fcurve3, label = ("'rFFT' series fit (%d terms)" % terms3)) # file 1
# ax.plot(x[0:17341], Fcurve3[0:17341], label = ("'rFFT' series fit (%d terms)" % terms3)) # bl cam
# ax.plot(x[0:18644], Fcurve3[0:18644], label = ("'rFFT' series fit (%d terms)" % terms3)) # file 2 with 7 sigma
# ax.plot(x[0:18646], Fcurve3[0:18646], label = ("'rFFT' series fit (%d terms)" % terms3)) # file 2
# ax.plot(x, Fcurve3, label = ("'rFFT' series fit (%d terms)" % terms3))
# ax.plot(x[0:16980], Fcurve3[0:16980], label = ("'rFFT' series fit (%d terms)" % terms3)) # file 3

ax.grid(True, color='dimgray', linestyle='--', linewidth=0.5)
ax.set_axisbelow(True)
ax.set_ylabel('differential magnitude')
ax.set_xlabel('time -BJDs')
ax.legend()

# Visualization
fig, ax = plt.subplots()
# Let's define a title for the figure.
# fig.suptitle("BL-Cam Power series - Sector 15, 2 Min Cadence")
fig.suptitle("XX-Cyg Power series - Sector 15, 2 Min Cadence")

ax.semilogy(freq[peaks_p], Pxx[peaks_p], 'x', label = ("'power' series peaks" )) #[0:600]
ax.semilogy(freq, Pxx, label = ("'power' series fit (%d terms)" % terms3), color='dimgray', linestyle='-', linewidth=0.5)

# ax.plot(freq[peaks_p], Pxx[peaks_p], 'x', label = ("'power' series peaks" )) #[0:600]
# ax.plot(freq, Pxx, label = ("'power' series fit (%d terms)" % terms3))

ax.grid(True, color='dimgray', linestyle='--', linewidth=0.5)
ax.set_axisbelow(True)
ax.set_ylabel('power')
ax.set_xlabel('frequency')
ax.legend()

print('Times of max light: ', x[peaks] + 2457000)
print('Main frequencies: ', freq[peaks_p])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Times of max light:  [2458738.69716034 2458738.83188205 2458738.96660373 2458739.10132538
 2458739.236047   2458739.37215749 2458739.50687909 2458739.64160067
 2458739.77632225 2458739.91104381 2458740.04576537 2458740.18048692
 2458740.31520847 2458740.44993001 2458740.58604043 2458740.72076196
 2458740.85548349 2458740.99020502 2458741.12492654 2458741.25964806
 2458741.39436957 2458741.52909108 2458741.66381259 2458741.79992298
 2458741.93464448 2458742.06936598 2458742.20408748 2458742.33880897
 2458742.47353046 2458742.60825195 2458742.74297344 2458742.87769492
 2458743.01241641 2458743.14852677 2458743.28324825 2458743.41796972
 2458743.5526912  2458743.68741267 2458743.82213414 2458743.95685561
 2458744.09157707 2458744.22629854 2458744.36240888 2458744.49574147
 2458744.63185181 2458744.76657327 2458744.90129473 2458745.03601619
 2458745.17073764 2458745.3054591  2458745.44018056 2458745.57490201
 2458745.70962347 2458745.84573381 2458745.98045527 2458746.11517672
 2458746.2498

In [11]:
from astropy.time import Time
times_of_max_light = Time(x[peaks] + 2457000, format='jd', scale='utc')
print(times_of_max_light.fits)

['2019-09-12T04:43:54.654' '2019-09-12T07:57:54.609'
 '2019-09-12T11:11:54.562' '2019-09-12T14:25:54.512'
 '2019-09-12T17:39:54.461' '2019-09-12T20:55:54.407'
 '2019-09-13T00:09:54.353' '2019-09-13T03:23:54.298'
 '2019-09-13T06:37:54.242' '2019-09-13T09:51:54.185'
 '2019-09-13T13:05:54.128' '2019-09-13T16:19:54.070'
 '2019-09-13T19:33:54.012' '2019-09-13T22:47:53.953'
 '2019-09-14T02:03:53.893' '2019-09-14T05:17:53.834'
 '2019-09-14T08:31:53.774' '2019-09-14T11:45:53.713'
 '2019-09-14T14:59:53.653' '2019-09-14T18:13:53.592'
 '2019-09-14T21:27:53.531' '2019-09-15T00:41:53.469'
 '2019-09-15T03:55:53.408' '2019-09-15T07:11:53.345'
 '2019-09-15T10:25:53.283' '2019-09-15T13:39:53.221'
 '2019-09-15T16:53:53.158' '2019-09-15T20:07:53.095'
 '2019-09-15T23:21:53.032' '2019-09-16T02:35:52.969'
 '2019-09-16T05:49:52.905' '2019-09-16T09:03:52.841'
 '2019-09-16T12:17:52.777' '2019-09-16T15:33:52.713'
 '2019-09-16T18:47:52.648' '2019-09-16T22:01:52.584'
 '2019-09-17T01:15:52.519' '2019-09-17T04:29:5

In [12]:
toml = Table()
toml["tBJD"] = x[peaks]
toml['HJD'] = times_of_max_light
toml['fits'] = times_of_max_light.fits
# toml.write('xxcyg_toml_4.fits', overwrite=True)
# toml.write('xxcyg_toml_4.csv', overwrite=True)
# toml.write('blcam_toml_1.fits', overwrite=True)
# toml.write('blcam_toml_1.csv', overwrite=True)
lc_xxcyg = toml
lc_xxcyg['dMag'] = lc_june.flux[peaks]
# toml.write('xxcyg_lc_4.csv', overwrite=True)
# toml.write('blcam_lc_1.csv', overwrite=True)

toml.show_in_notebook()

idx,tBJD,HJD,fits,dMag
0,1738.697160343795,2458738.6971603436,2019-09-12T04:43:54.654,1.399987901670727
1,1738.8318820539157,2458738.831882054,2019-09-12T07:57:54.609,1.3910121284506185
2,1738.966603728531,2458738.9666037285,2019-09-12T11:11:54.562,1.3777578893976363
3,1739.1013253760198,2458739.101325376,2019-09-12T14:25:54.512,1.38245112260272
4,1739.2360470020908,2458739.236047002,2019-09-12T17:39:54.461,1.38409936870226
5,1739.372157493723,2458739.3721574936,2019-09-12T20:55:54.407,1.3996783827161285
6,1739.5068790889413,2458739.506879089,2019-09-13T00:09:54.353,1.3958997037721574
7,1739.6416006727532,2458739.6416006726,2019-09-13T03:23:54.298,1.39517066910065
8,1739.7763222469016,2458739.776322247,2019-09-13T06:37:54.242,1.3998006311752675
9,1739.911043812786,2458739.911043813,2019-09-13T09:51:54.185,1.400409395842919


In [13]:
times = Time(x + 2457000, format='jd', scale='utc')
print(len(np.array(lc_june.flux)))
ts = TimeSeries(data= [lc_june.flux], time=times)
period = 0.134865113 * u.d
epoch = times_of_max_light[0]
ts_f = ts.fold(period=period, midpoint_epoch = epoch) # , epoch_time = epoch)  
print(ts_f.colnames)

# Visualization
fig, ax = plt.subplots()

fig.suptitle("XX-Cyg Folded Series - Sector 15, 2 Min Cadence")
# print(ts_f['col0'])
ax.scatter(ts_f['time'].jd, ts_f['col0'], label = ('Folded LightCurve'), s = 0.1)

ax.grid(True, color='dimgray', linestyle='--', linewidth=0.5)
ax.set_axisbelow(True)
ax.set_ylabel('Flux')
ax.set_xlabel('Time')
ax.legend()


16834
['time', 'col0']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# lc_periodogram = lc_june.to_periodogram(method = 'bls')
# # lc_periodogram.plot()

# # Print information
# print(lc_periodogram.max_power)
# print(lc_periodogram.frequency_at_max_power)
# print(lc_periodogram.period_at_max_power)
# print(lc_periodogram.depth_at_max_power)
# print(lc_periodogram.duration_at_max_power)
# print(lc_periodogram.transit_time_at_max_power)
# # Ignore lack of specification warnings !!

# # Retrieve approximate lightcurve fit 
# # This is built for exoplanet transits and box like light curve features
# lc_fit = lc_periodogram.get_transit_model()

# ax = lc_fit.plot()
# lc_june.scatter(ax = ax, s = 0.01,marker = '.', c = 'b', label = 'XX-Cyg lightcurve')



In [15]:
# Print the end time and total computing time
END_DATE_TIME = datetime.datetime.now()
print('\nEnding time: ', END_DATE_TIME)
print("Time elapsed: ", (END_DATE_TIME - START_DATE_TIME))


Ending time:  2020-03-02 20:00:47.874753
Time elapsed:  0:00:37.517623


## References
[1]: Thomas E. Obert, Joseph E. Rodriguez, Knicole D Colon, et al. 	arXiv:1608.00618 [astro-ph.EP] https://arxiv.org/abs/1608.00618

[2]: Kelt survey, https://keltsurvey.org/planets/kelt-16b

[3]: Gudmundur Stefansson, Astrobites | Sep 20, 2016, "KELT-16b: a new benchmark for future exoplanet atmosphere studies", https://astrobites.org/2016/09/20/kelt-16b-a-new-benchmark-for-future-exoplanet-atmosphere-studies/

[4]: Data Validation report for TESS ID 236445129 sectors 15-15 21-sep-2019 11:23:33 Z